In [1]:
DOMAIN = 'test' # test, spouse, semparse_cdr

In [2]:
import sys
import os

os.environ['SNORKELDB'] = 'postgres:///{}'.format(DOMAIN)

from snorkel import SnorkelSession
session = SnorkelSession()

from snorkel.models import candidate_subclass
if DOMAIN in ['test', 'spouse']:
    Spouse = candidate_subclass('Spouse', ['person1', 'person2'])
    candidate_class = Spouse
elif DOMAIN == 'semparse_cdr':
    ChemicalDisease = candidate_subclass('ChemicalDisease', ['chemical', 'disease'])
    candidate_class = ChemicalDisease
else:
    raise Exception("Invalid DOMAIN.")

In [3]:
candidates = session.query(candidate_class).all()
print("Candidates: {}".format(len(candidates)))

Candidates: 45


In [4]:
from snorkel.semantic import (get_left_phrases, get_right_phrases, 
                              get_between_phrases, get_sentence_phrases)
c = candidates[1]
print c[0].get_parent()._asdict()['text']
print c

David Beckham already had four other perfumes, two of which also had a women's version fronted by his lovely wife Victoria Beckham. 
Spouse(Span("Victoria Beckham", sentence=673, chars=[114,129], words=[22,23]), Span("David Beckham", sentence=673, chars=[0,12], words=[0,1]))


In [5]:
for p in get_left_phrases(c[0], cmp='.eq', num=1, n_max=1):
    print getattr(p, 'text')

wife


In [6]:
# c[0].get_parent()._asdict()

In [7]:
# from snorkel.viewer import SentenceNgramViewer
# sv = SentenceNgramViewer(candidates[:300], session)
# sv

In [8]:
# c = sv.get_selected()
# print c
# print c[0].get_parent()._asdict()['pos_tags']
# print c[0].get_parent()._asdict()['ner_tags']

In [9]:
# hash(c)

In [10]:
from semparse_examples import get_examples

examples = get_examples(DOMAIN, candidates)

if DOMAIN == 'test':
    user_lists = {
        'colors':['red','green','blue'],
        'bluebird':['blue','bird','fly'],
        'greek':['alpha','beta','gamma'],
        'letters':['a','B','C'],
        'smalls':['a','b','c','d'],
        'spouse':['wife','husband','spouse']}
elif DOMAIN == 'spouse':
    user_lists = {
        'spouse':['wife','husband','spouse'],
        'family':['father', 'mother', 'brother', 'sister']}
elif DOMAIN == 'semparse_cdr':
    user_lists = {}

In [11]:
# examples[0].display()

In [12]:
# examples[28].candidate[0].get_parent()

In [13]:
%time from snorkel.semantic import SemanticParser

CPU times: user 14 µs, sys: 22 µs, total: 36 µs
Wall time: 17.9 µs


In [14]:
%time sp = SemanticParser(candidate_class, user_lists, absorb=False)

Created grammar with 246 rules
CPU times: user 4.45 ms, sys: 6.07 ms, total: 10.5 ms
Wall time: 12.8 ms


In [15]:
%time results = sp.evaluate(examples, show_explanation=True, show_all=False, only=[])
results

Example 0: label True because True
Example 1: label True because True and True
Example 2: label True because False or True
Example 3: label True because not False
Example 4: label True because 'yes' equals 'yes'
Example 5: label True because 1 is equal to 1
Example 6: label True because 1 is equal to one
Example 7: label True because 1 is less than 2
Example 8: label True because 2 is less than or equal to 2
Example 9: label True because 2 > 1
Example 10: label True because 2 is at least 2
Example 11: label True because arg 1 is lowercase
Example 12: label True because arg 1 is upper case
Example 13: label True because arg 1 is capitalized
Example 14: label True because the word 'blueberry' starts with 'blue'
Example 15: label True because the word 'blueberry' ends with 'berry'
Example 16: label True because argument 1 equals 'foo'
Example 17: label True because 'bar' is in 'foobarbaz'
Example 18: label True because the word 'foobarbaz' contains 'oobarba'
Example 19: label True because

,Correct,Incorrect,Redundant,Failed,Unknown
Example0,1,0,0,0,0
Example1,1,0,0,0,0
Example2,1,0,0,0,0
Example3,1,0,0,0,0
Example4,1,0,0,0,0
Example5,1,0,0,0,0
Example6,1,0,0,0,0
Example7,1,0,0,0,0
Example8,1,0,0,0,0
Example9,1,0,0,0,0


In [16]:
import numpy as np
print np.sum(results, axis=0)

Correct      55
Incorrect     8
Redundant     7
Failed        0
Unknown       0
dtype: int64


In [17]:
# sp.grammar.print_chart(nested=True)

In [18]:
# sp.grammar.print_grammar()